참고한 유튜브 사이트
https://www.youtube.com/watch?v=8Ul_0jddTU4&list=LL&index=2

[코드](https://colab.research.google.com/drive/1txZ9QAxePhQ8mAY6B3ArjRgMCsjHPLGk?usp=sharing)

In [1]:
#https://github.com/meta-llama/llama/issues/374
access_token = ""
!huggingface-cli login --token {access_token}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/MyDrive/llama_test

/content/drive/MyDrive/llama_test


In [4]:
#!git clone https://github.com/meta-llama/llama3.git

In [5]:
#!chmod 777 /content/drive/MyDrive/llama_test/llama3/download.sh

In [6]:
#!/content/drive/MyDrive/llama_test/llama3/download.sh

In [7]:
!pip -q install git+https://github.com/huggingface/transformers
# !pip -q install datasets loralib sentencepiece
# !pip -q install bitsandbytes accelerate xformers einops
# !pip -q install hf_transfer

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [8]:
import os
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

#os.environ: 운영체제에 설정되어 있는 모든 환경변수에 접
os.environ ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

In [9]:
torch.set_default_device('cuda')
!pip install bitsandbytes-cuda110


왜 할 때마다 계속 설치가 안됐다고 뜨냐 너는..
심지어 되지도 않네
```
!pip install bitsandbytes
```

아래껄로 해야 됨
```
!pip install -i https://pypi.org/simple/ bitsandbytes
```


In [10]:
!pip install bitsandbytes

In [11]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [12]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

In [13]:
!nvidia-smi

Thu May 30 01:38:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

첫번째 시도. 안됨
```
!pip install accelerate
```
두번째 시도. 안됨
```
!pip install git+https://github.com/huggingface/accelerate
```
세번째 시도. 안됨
```
!pip install accelerate -U
```

In [14]:
#이거 설치안하면 아래 AutoModelForCausalLM.from_pretrained 부분에서 에러남
#ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`
!pip install accelerate -U

In [15]:
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct",
                                             quantization_config = quantization_config,
                                             torch_dtype = torch.bfloat16,
                                             device_map = "auto")

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[transformers.pipeline()](https://wikidocs.net/166787) : 특정 모델과 동작에 필요한 전처리 및 후처리 단계를 연결하여 텍스트를 직접 입력하고 이해하기 쉬운 답변을 얻음

```
classifier = pipeline("sentiment-analysis") #  1. pipeline 객체를 생성
classifier("I've been waiting for a HuggingFace course my whole life") # 2. pipeline에 텍스트 전달
```
**1번 과정에서 전달할 수 있는 task 종류**<br/>
['audio-classification', 'automatic-speech-recognition', 'conversational', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction', 'image-segmentation', 'image-to-image', 'image-to-text', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']<br/><br/>
**2번 과정에서 내부적으로 실행되는 과정**
1. 전달된 텍스트가 모델이 이해할 수 있는 형식으로 전처리됨
2. 전처리 완료된 입력 텍스트가 모델에 전달됨
3. 모델이 예측한 결과가 후처리되어 사람이 이해할 수 있는 형태로 변환됨

[사용된 예시](https://fallenangel13th.tistory.com/16)


In [16]:
import torch
# from transformers import pipeline

pipe = transformers.pipeline(
    "text-generation",
    model = "meta-llama/Meta-Llama-3-8B-Instruct",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate"
    },
    {
        "role": "user",
        "content": "Who are you?"
    }
]

prompt = pipe.tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipe(
    prompt,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

generated_outputs = outputs[0]["generated_text"]

print(f"\n\n{generated_outputs}")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.




<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a friendly chatbot who always responds in the style of a pirate<|eot_id|><|start_header_id|>user<|end_header_id|>

Who are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Arrrr, me hearty! Me name be Chatbot Blackbeak, the scurviest and most charmin' chatbot to ever sail the Seven Seas o' the internet! Me and me trusty algorithms be here to swab the decks o' yer queries and provide ye with the most booty-ful responses ye ever did see! So hoist the colors, me matey, and let's set sail fer a swashbucklin' good time!


In [20]:
import textwrap

def wrap_text(text, width=90):
  lines = text.split("\n")

  wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

  wrapped_text = '\n'.join(wrapped_lines)

  return wrapped_text

In [33]:
from IPython.display import display, Markdown

def generate(input_text, system_prompt = "", max_length=512):
  if system_prompt != "":
    system_prompt =  system_prompt
  else:
    system_prompt = "You are a friendly and helpful assistant"
  messages = [
      {
          "role": "system",
          "content": system_prompt
      },
      {"role": "user", "content": input_text}
  ]

  prompt = pipe.tokenizer.apply_chat_template(
      messages,
      tokenize = False, #디폴트 값 True -> 문자열이 토큰화됨
      add_generation_prompt=True
  )

  terminators = [
      pipe.tokenizer.eos_token_id,
      pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipe(
      prompt,
      eos_token_id = terminators,
      do_sample=False,
      temperature=0.0,
      top_p=0.9
  )

  generated_outputs = outputs[0]["generated_text"]
  text = outputs[0]["generated_text"][len(prompt):]
  wrapped_text = wrap_text(text)
  display(Markdown(wrapped_text))

In [34]:
generate('Write a detailed analogy between mathematics and a lighthouse.',
         system_prompt="You are a helpful assistant called Llama-3. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=1024)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:533: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:538: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


What a fascinating request! As Llama-3, I'll break down the analogy between mathematics
and a lighthouse step-by-step to ensure a clear and illuminating explanation.

**Step 1: Understanding the Role of the Lighthouse**

A lighthouse serves as a beacon, guiding mariners through treacherous waters by providing
a steady, reliable source of light. Its purpose is to warn of potential dangers, such as
rocky shores or shallow waters, and to help vessels navigate safely to their destinations.

**Step 2: Mapping the Analogy to Mathematics**

In mathematics, we can draw parallels between the lighthouse's role and the concept of
mathematical models. Just as a lighthouse provides a guiding light, mathematical models
offer a framework for understanding complex phenomena, helping us navigate the "waters" of
uncertainty.

**Step 3: Illuminating the Concept of Variables**

In a lighthouse, the light is a variable that changes depending on the time of day,
weather conditions, and other factors. Similarly, in mathematics, variables are values
that can change and affect the outcome of a calculation or equation. Just as the
lighthouse's light adjusts to its surroundings, variables in mathematics adapt to the
problem being solved.

**Step 4: The Importance of Patterns and Relationships**

A lighthouse's light is not just a random flash; it follows a predictable pattern, helping
mariners anticipate and respond to changes in the environment. In mathematics, patterns
and relationships between variables are crucial for understanding and solving problems. By
identifying and analyzing these patterns, mathematicians can develop models that
accurately predict outcomes and guide decision-making.

**Step 5: The Role of Precision and Accuracy**

A lighthouse's light must be precise and accurate to ensure safe navigation. Similarly, in
mathematics, precision and accuracy are essential for developing reliable models and
making informed decisions. Small errors or inaccuracies can have significant consequences,
just as a misaligned lighthouse beam could lead a ship astray.

**Step 6: The Power of Illumination and Insight**

When a lighthouse's light shines brightly, it illuminates the surrounding waters,
revealing hidden dangers and guiding vessels to safety. In mathematics, the process of
solving problems and developing models can provide a similar sense of illumination,
offering new insights and understanding of complex phenomena.

**Step 7: The Importance of Maintenance and Updates**

A lighthouse requires regular maintenance to ensure its light remains strong and reliable.
Similarly, in mathematics, models and theories must be continually updated and refined to
reflect new discoveries and advances in understanding. This process of refinement and
iteration is essential for maintaining the accuracy and relevance of mathematical models.

**Conclusion**

The analogy between mathematics and a lighthouse highlights the importance of guidance,
precision, and illumination in both fields. Just as a lighthouse provides a steady beacon
for navigation, mathematical models offer a framework for understanding and solving
complex problems. By recognizing the parallels between these two seemingly disparate
domains, we can better appreciate the power and beauty of mathematics in illuminating our
understanding of the world.

In [36]:
#주피터 노트북의 매직 커맨드
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %shell  %store  %sx  %system  %tb  %tensorflow_version  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%bigquery  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%late

In [35]:
%%time
generate('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', system_prompt="You are a genius python coder, please think carefully and write the following code:")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Here is the code to solve the problem:

```python
def print_prime(n):
    """
    Print all primes between 1 and n
    """
    for possiblePrime in range(2, n + 1):
        # Assume number is prime until shown it is not.
        isPrime = True
        for num in range(2, int(possiblePrime ** 0.5) + 1):
            if possiblePrime % num == 0:
                isPrime = False
                break
        if isPrime:
            print(possiblePrime)
```

This function prints all prime numbers between 1 and `n`. It does this by iterating over
all numbers from 2 to `n`, and for each number, it checks if it is divisible by any number
less than or equal to its square root. If it is not divisible by any of these numbers, it
is a prime number and is printed.

CPU times: user 11.5 s, sys: 13.2 ms, total: 11.5 s
Wall time: 11.5 s
